In [ ]:
import os
import json
import pandas as pd
import re

# 데이터 경로 설정
train_audio_dir = ""
train_json_dir = ""
valid_audio_dir = ""
valid_json_dir = ""

# JSON 파일 로드 함수
def load_json_files(directory):
    json_data = []
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as f:
                data = json.load(f)
                json_data.append(data)
    return json_data

# 데이터프레임 생성
def create_dataframe(json_data, audio_dir):
    data = []
    for item in json_data:
        file_name = item.get('fileName')
        answer_text = item.get('transcription', {}).get('AnswerLabelText', '')
        audio_path = os.path.join(audio_dir, file_name)
        if os.path.exists(audio_path) and answer_text:
            data.append({
                'file_path': audio_path,
                'text': answer_text
            })
    return pd.DataFrame(data)

# 학습 및 검증 데이터 JSON 로드
train_json = load_json_files(train_json_dir)
valid_json = load_json_files(valid_json_dir)

# 학습 및 검증 데이터프레임 생성
train_df = create_dataframe(train_json, train_audio_dir)
valid_df = create_dataframe(valid_json, valid_audio_dir)

# 라벨 전처리 함수 (공백 포함, 한글+공백+기본 구두점만 남김)
def prepare_korean_text(text):
    text = re.sub(r'[^\uAC00-\uD7A3\s.,!?]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

train_df['normalized_text'] = train_df['text'].apply(prepare_korean_text)
valid_df['normalized_text'] = valid_df['text'].apply(prepare_korean_text)

print(f"학습 데이터 크기: {len(train_df)}")
print(f"검증 데이터 크기: {len(valid_df)}")
print(train_df[['text', 'normalized_text']].head())
